In [4]:
from regain.datasets.hmm import generate_hmm
from regain.hmm.incremental_hmm_graphical_lasso import Incremental_HMM_GraphicalLasso as IncHMM
import numpy as np
from importlib import reload
import regain.hmm.utils; reload(regain.hmm.utils)
from regain.hmm.utils import results_recap#,prepare_data_to_predict
from regain.hmm.utils import cross_validation
from regain.hmm.hmm_graphical_lasso import HMM_GraphicalLasso
import matplotlib.pyplot as plt

/Users/federicociech/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/federicociech/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
N_samples = 1000
N_states = 5
N_TS = 10
sigma_Normal = 1


In [ ]:
Ntrial =8

res_Vmeas_batch = 0
res_MCC_batch = 0

lists_perc_increm = [0.025,0.05,0.075,0.1, 0.15,0.2,0.25,0.3]

res_Vmeas_Inc = np.zeros(len(lists_perc_increm))
res_MCC_Inc = np.zeros(len(lists_perc_increm))

for n in range(Ntrial):


    dataset = generate_hmm(n_samples= N_samples,
                           n_states=N_states,
                           n_dim_obs=N_TS,
                           mode_precisions='regain',
                           mode_mean = 'Normal',
                           transition_type='fixed',
                           sigma=sigma_Normal)

    lambda_GL = 30
    toll = 1e-4
    max_iterations = 100
    reps = 10

        
    # Batch
    
    hmm_gmm = HMM_GraphicalLasso(n_clusters=N_states,
                             alpha=lambda_GL,
                             tol=toll,
                             verbose=False,
                             mode='scaled',
                             warm_restart=True,
                             max_iter=max_iterations,
                             repetitions=reps,
                             n_jobs=-1)
    hmm_gmm.fit(dataset['data'])
    res_batch = results_recap(dataset['states'], hmm_gmm.labels_, dataset['thetas'], hmm_gmm.precisions_, 
                  dataset['gammas'], hmm_gmm.gammas_)
    
    res_Vmeas_batch += res_batch['v_measure [0, 1]']/Ntrial
    res_MCC_batch += res_batch['weighted_mean_mcc [-1, 1]'][0]/Ntrial
    
    
    # Incremental

    for ii,perc_increm in enumerate(lists_perc_increm):
        mdl = IncHMM(n_clusters=N_states,
                     alpha=lambda_GL,
                     n_for_init=int(N_samples*(1-perc_increm)),
                     max_iter=max_iterations,
                     tol=toll,
                     verbose=False,
                     warm_restart=True,
                     repetitions=reps,
                     n_jobs=-1)
        
        mdl.fit(dataset['data'])
        res_increm = results_recap(dataset['states'], mdl.labels_, dataset['thetas'], mdl.precisions_, 
                      dataset['gammas'], mdl.gammas_)
        
        res_Vmeas_Inc[ii] = res_Vmeas_Inc[ii] + res_increm['v_measure [0, 1]']/Ntrial
        res_MCC_Inc[ii] = res_MCC_Inc[ii] + res_increm['weighted_mean_mcc [-1, 1]'][0]/Ntrial

In [5]:
resV_tot = []
resV_tot.append(res_Vmeas_batch)
resV_tot.extend(res_Vmeas_Inc)

In [27]:
resM_tot = []
resM_tot.append(res_MCC_batch)
resM_tot.extend(res_MCC_Inc)